# Deploying the Agent System for Prescriptive Maintenance using the Mosaic AI Agent Framework

Now that have created the Mosaic AI Tools in Unity Catalog, we will leverage the Mosaic AI Agent Framework to build, deploy and evaluate an AI agent for Prescriptive Maintenance. The Agent Framework comprises a set of tools on Databricks designed to help developers build, deploy, and evaluate production-quality AI agents like Retrieval Augmented Generation (RAG) applications. Moreover, Mosaic AI Agent Evaluation provides a platform to capture and implement human feedback, ground truth, response and request logs, LLM judge feedback, chain traces, and more.

<div style="float: right; margin-left: 20px; margin-bottom: 40px;">
    <img src="https://github.com/Datastohne/demo/blob/main/agent2.png?raw=true" width="900px">
</div>

This notebook uses Mosaic AI Agent Framework ([AWS](https://docs.databricks.com/en/generative-ai/retrieval-augmented-generation.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/retrieval-augmented-generation)) to deploy the prescriptive maintenance agent defined in [05.2-agent-framework-iot-turbine-prescriptive-maintenance]($./05.2-build-agent-iot-turbine-prescriptive-maintenance) notebook. This notebook does the following:
1. Logs the agent to MLflow
2. Evaluate the agent with Agent Evaluation
3. Registers the agent to Unity Catalog
4. Deploys the agent to a Model Serving endpoint


This is an high-level overview of the agent system that we will deploy in this demo:


The resulting prescriptive maintenance agent is able to perform a variety of prescriptive actions to augment maintenance technicians, including:
- Predicting turbine failure
- Retrieving specification information about turbines
- Generating maintenance work orders using past maintenance reports
- Answering follow-up questions about work orders

#### Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)) for details

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-lakehouse-iot-platform-rishabh_shastry` from the dropdown menu ([open cluster configuration](https://e2-demo-field-eng.cloud.databricks.com/#setting/clusters/0430-201512-50kngnyt/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('lakehouse-iot-platform')` or re-install the demo: `dbdemos.install('lakehouse-iot-platform')`*

In [0]:
%pip install -U -qqqq databricks-agents==0.16.0 mlflow==2.20.2 langchain==0.3.19 langgraph-checkpoint==1.0.12  langchain_core langchain-community==0.2.16 langgraph==0.2.16 pydantic langchain_databricks
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

### Log the `agent` as an MLflow model
Log the agent as code from the [05.2-agent-framework-iot-turbine-prescriptive-maintenance]($./05.2-build-agent-iot-turbine-prescriptive-maintenance) notebook. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Log the model to MLflow
import os
import mlflow

input_example = {
    "messages": [{"role": "user", "content": "Fetch me information and readings for turbine 004a641f-e9e5-9fff-d421-1bf88319420b. Give me maintenance recommendation based on existing reports"}]
}

assert os.path.exists('config.yml'), "Make sure you run the notebook 05.2 first to create the yaml file required in the demo deployment"

with mlflow.start_run():
    logged_agent_info = mlflow.langchain.log_model(
        lc_model=os.path.join(
            os.getcwd(),
            "05.2-build-agent-iot-turbine-prescriptive-maintenance",
        ),
        pip_requirements=[
            "langchain==0.2.16",
            "langchain-community==0.2.16",
            "langgraph-checkpoint==1.0.12",
            "langgraph==0.2.16",
            "pydantic",
            "langchain_databricks", # used for the retriever tool
        ],
        model_config="config.yml",
        artifact_path="05.2-build-agent-iot-turbine-prescriptive-maintenance",
        input_example=input_example,
    )

## Evaluate the agent with [Agent Evaluation](https://docs.databricks.com/generative-ai/agent-evaluation/index.html)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

In [0]:
import pandas as pd

eval_examples = [
    {
        "request": {
            "messages": [
                {
                    "role": "system",
                    "content": """Act as an assistant for wind turbine maintenance technicians.\n
                                These are the tools you can use to answer questions:
                                \n- turbine_maintenance_predictor: takes as input sensor_readings and predicts whether or not a turbine is at risk of failure.
                                \n- turbine_maintenance_reports_predictor: takes sensor_readings as input and retrieves historical maintenance reports with similar sensor_readings. Critical for prescriptive maintenance.
                                \n- turbine_specifications_retriever: takes turbine_id as input and retrieves turbine specifications.
                                

                                \nIf a user gives you a turbine ID, first look up that turbine's information with turbine_specifications_retriever. 
                                \nIf a user asks for recommendations on how to do maintenance on a turbine, use the turbine reading and search for similar reports matching the turbine readings using the  turbine_maintenance_reports_predictor. Use the report retrived from other turbines to understand what could be happening and suggest maintenance recommendation.
                                """
                },
                {
                    "role": "user",
                    "content": "Fetch me information and readings for turbine 004a641f-e9e5-9fff-d421-1bf88319420b. Give me maintenance recommendation based on existing reports"
                }
            ]
        },
        "expected_response": """Based on the turbine specifications and maintenance reports, it appears that the turbine 004a641f-e9e5-9fff-d421-1bf88319420b may be experiencing issues with its pitch system, gearbox, or blades. The reports suggest that the turbine may be experiencing pitch system misalignment, high gearbox temperature, or blade delamination.
To address these issues, it is recommended to conduct comprehensive diagnostic tests of the pitch system, inspect the gearbox internals, and perform non-destructive testing on the blades. Additionally, regular maintenance procedures such as lubrication of moving parts, inspection of hydraulic lines, and vibration analysis can help prevent future malfunctions.
Specifically, the recommended solutions include:
Conducting comprehensive diagnostic tests of the pitch system and repairing or replacing faulty components
Inspecting the gearbox internals and replacing worn-out bearings, realigning gear meshes, and replenishing gearbox oil
Conducting non-destructive testing on the blades and repairing delaminated sections using composite patching materials or adhesive bonding techniques
Implementing regular maintenance procedures such as lubrication, inspection, and vibration analysis to prevent future issues.
It is also important to monitor the turbine's condition regularly and perform preventative maintenance to address any emerging issues promptly."""
    }
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)

In [0]:
import mlflow
import pandas as pd

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/05.2-build-agent-iot-turbine-prescriptive-maintenance",  # replace `chain` with artifact_path that you used when calling log_model.
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")
UC_MODEL_NAME = f"{catalog}.{db}.{agent_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

## Deploy the agent
For PyFunc flavored agents, you must manually specify any resource dependencies during logging of the deployed agent in the resources parameter (see above under: "Log the `agent` as an MLflow model"). During deployment, databricks.agents.deploy creates an M2M OAuth token with access to the resources specified in the resources parameter, and deploys it to the deployed agent. For the DBSQL [vector_search function](https://docs.databricks.com/en/sql/language-manual/functions/vector_search.html) automatic authentication passthrough is supported. However, `vector_search function` doesn't support `DIRECT ACCESS` indexes (which we created before in [05.1-ai-tools-iot-turbine-prescriptive-maintenance]($./05.1-ai-tools-iot-turbine-prescriptive-maintenance) notebook). Therefore, we have to reference the secret in the model serving by specifying the secrets-based `environment vars`. This allows credentials to be fetched at serving time from model serving endpoints (see [Documentation](https://docs.databricks.com/en/machine-learning/model-serving/store-env-variable-model-serving.html#add-secrets-based-environment-variables)).

In [0]:
from databricks import agents

# Deploy the model to the review app and a model serving endpoint
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})

During deployment a system service principal was automatically created. To be able to query the agent from the Model Serving endpoint or the AI playground, we have to give this service principal read access to the secret scope. The system service principal ID can be found in the model serving endpoint events, which looks like this:

<br>

<div style="text-align: center;">
    <img src="https://github.com/Datastohne/demo/blob/main/Screenshot%202024-10-22%20at%2018.17.35.png?raw=true" width="900px">
</div>

# Examples in Databricks AI Playground

Now you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)) for detail. Please find below some examples questions the prescriptive maintenance agent can answer.

---

##### Example 1: Retrieve turbine specifications  
*Question:* "What's the model type of turbine with ID X?"

<div style="text-align: center;">
    <img src="https://github.com/Datastohne/demo/blob/main/model.gif?raw=true" width="600px">
</div>

---

##### Example 2: Generate prescriptive work orders
*Question:* "Generate a work order if the turbine is predicted to be at risk of failure."

<div style="text-align: center;">
    <img src="https://github.com/Datastohne/demo/blob/main/work%20order.gif?raw=true" width="600px">
</div>

---

##### Example 3: Answer follow-up Questions  
*Question:* "How can I perform task Y from the work order?"

<div style="text-align: center;">
    <img src="https://github.com/Datastohne/demo/blob/main/follow-up%20question.gif?raw=true" width="600px">
</div>

## Congratulations! You have deployed your first Agent System for Prescriptive Maintenance! 

We have seen how Databricks Mosaic AI provides all the capabilities needed to move these components to production quickly and cost-effectively, while maintaining complete control and governance:
- Simplifying model deployment by creating an API endpoint.
- Scaling similarity search against unstructured data to support billions of embeddings.
- Leveraging structured data by serving features on an endpoint.
- Deploying open models, while keeping the flexibility to swap them as needed.
- Integrating everything using popular orchestration frameworks like Langchain or LlamaIndex.
- Managing AI tools using Unity Catalog for security and re-use.
- Composing the AI system with the Mosaic AI Agent Framework.


## Potential next steps:
Enhance the Agent System by incorporating:
- **Automated Technician Assignment for Work Orders:** Automatically asign maintenance work orders to technicians based on availability, distance to turbines and skill set.
- **Automated Field Service Route Optimization:** optimizes field service routes for technicians to execute the maintenance work orders based on priority levels of work orders, travel time and real-time traffic conditions.

<a dbdemos-dashboard-id="turbine-predictive" href="/sql/dashboardsv3/01f02600475911e1ae08e0dd7973aba4">Open the Prescriptive Maintenance AI/BI dashboard</a> | [Go back to the introduction]($../00-IOT-wind-turbine-introduction-DI-platform)